In [1]:
%load_ext autoreload
%autoreload 2
import sys, os, torch, datetime, time
import numpy as np
sys.path.append("..")
import rfi_ml
import matplotlib.pyplot as plt

In [2]:
Ntrain = 100000*30 #the size of the training set
Ntest = 10
sigma_low = 20
sigma_high = 50
Nepochs = 30
Nrfi = 1
Np = 2**10 #1k
z_dim = 16
Nfft = Np // 2 + 1
Pk = np.ones(Nfft)
#lamb = 0.9 #lambda, scaling down Gaussianized amplitude
np.random.seed(0) #for debugging
torch.manual_seed(0) #more debugging
start_time = time.time()

t = rfi_ml.ToyGenerator(Np=Np)
g = torch.stack([torch.from_numpy(t.getGaussian()) for i in range(Ntrain+Ntest)])
rfi_array = np.zeros((Ntrain+Ntest, Np), dtype=float)
for i in range(Ntrain + Ntest):
    rfi_timestream = np.zeros(Np)
    for j in range(Nrfi):
        rfi_timestream += t.getNonGaussianLocalized(sigma=(sigma_low, sigma_high)) 
        #rfi_timestream += t.getNonGaussianLocalized(sigma=(sigma_low, sigma_high), ampl=(0.05, 0.05)) 
        #rfi_timestream += t.getNonGaussianNonlocalized(Pflip=1.0) 
    rfi_array[i,:] += rfi_timestream
ng = torch.from_numpy(rfi_array)
#ng = torch.stack([torch.from_numpy(t.getNonGaussianLocalized(sigma=(sigma_low, sigma_high))) for i in range(Ntrain+Ntest)])


RuntimeError: [enforce fail at ..\c10\core\CPUAllocator.cpp:75] data. DefaultCPUAllocator: not enough memory: you tried to allocate 24576081920 bytes. Buy new RAM!

In [ ]:
n = rfi_ml.RFIDetect(Np, Nepochs = Nepochs, z_dim = z_dim)

In [ ]:
n.train(g[:Ntrain], ng[:Ntrain])

In [ ]:
testout = n.evaluate(g[Ntrain:], ng[Ntrain:])

In [ ]:
n.plot_eval(testout, g[Ntrain:], ng[Ntrain:])

In [ ]:
print("Runtime: ", time.time()-start_time)